In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.ndimage.morphology import binary_closing, binary_fill_holes
from skimage import exposure
import closed_pores_evaluation as cpe
from file_paths import get_path
from pathlib import Path
from PIL import Image

In [ ]:
def get_2d_slice(num, file_id):
    data_folder = get_path(file_id)

    file_names = Path(data_folder).glob('*.tiff')
    file_names = list(file_names)
    img2d_gray = np.array(Image.open(file_names[num]))

    return img2d_gray

In [ ]:
file_id='123493'
fig, ax = plt.subplots(figsize=(10, 10))
num = 500 # 320
img2d_grey = get_2d_slice(num, file_id=file_id) # img3d[num]
ax.imshow(img2d_grey, cmap="gray")
ax.axis("off")

In [ ]:
img_equalized = exposure.equalize_hist(img2d_grey)

In [ ]:
img_equalized_binarized = img_equalized>0.5

mask = cpe.find_mask_longest_contours(~img_equalized_binarized,
                                      filter_by_contour_length=True,
                                      min_contour_length = 2500,
                                      max_number_of_contours=None
                                      )

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(20, 20), constrained_layout=True)
axes = axes.ravel()
for ax in axes:
    ax.axis("off")
axes[0].imshow(mask, cmap="gray")

axes[1].imshow(binary_fill_holes(mask), cmap="gray")
axes[1].set_title(f"contours were here")

axes[2].imshow(img2d_grey, cmap="gray")

mask_f = binary_fill_holes(mask)
axes[3].imshow(img2d_grey, cmap="gray")
axes[3].imshow(np.ma.masked_where(mask_f < 1, mask_f), cmap='hsv', alpha=0.3, interpolation='none')
axes[3].set_title('without_boarders')

# gradint trial

In [ ]:
file_id='123493'
fig, ax = plt.subplots(figsize=(10, 10))
num = 500 # 320
img2d_grey = get_2d_slice(num, file_id=file_id) # img3d[num]
ax.imshow(img2d_grey, cmap="gray")
ax.axis("off")

In [ ]:
fig.savefig('plots\colab_123493', bbox_inches='tight', pad_inches=0)

In [ ]:
img_equalized = exposure.equalize_hist(img2d_grey)

fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(img_equalized, cmap="gray")
ax.axis("off")

In [ ]:
from scipy.ndimage import sobel

In [ ]:
def gradient(img):
    sx = sobel(img, axis=0, mode='constant')
    sy = sobel(img, axis=1, mode='constant')
    return np.hypot(sx,sy)

In [ ]:
img_equalized_grad = gradient(img_equalized)
img_equalized_laplas = gradient(img_equalized_grad)

fig, axes = plt.subplots(ncols=3, nrows=2, figsize=(30, 20), constrained_layout=True)
axes[0, 0].imshow(img_equalized, cmap="gray")
axes[0, 1].imshow(img_equalized_grad, cmap="gray")
axes[0, 2].imshow(img_equalized_laplas, cmap="gray")

axes[1, 0].hist(img_equalized.flatten(), bins=255)
axes[1, 1].hist(img_equalized_grad.flatten(), bins=255)
axes[1, 2].hist(img_equalized_laplas.flatten(), bins=255)
# axes[0].axis("off")

In [ ]:
+2

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(exposure.equalize_hist(img_equalized_grad), cmap="gray")
ax.axis("off")

In [ ]:
coef = np.max(img_equalized_grad) - np.min(img_equalized_grad) 

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(img_equalized_grad>0.7*coef, cmap="gray")
ax.axis("off")